In [5]:
# app.py
"""
Gradio-based Water Quality Chatbot (Generative-style reasoning + Neural net prediction)

Features:
- Loads/trains a TensorFlow Keras model on 'data/water_potability.csv' (if model file missing).
- Extracts water parameters from free-text user queries via regex (spaCy optional).
- Predicts Potability probability and returns classification.
- Generates human-readable explanation and recommended actions (rule-based + data-driven).
- Provides parsed feature table for transparency.

Usage:
> python app.py
Then open the local Gradio URL shown in the terminal.
"""

import os
import re
import json
import numpy as np
import pandas as pd
import joblib
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# ML & DL
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# UI
import gradio as gr

# Optional NLP
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
except Exception:
    nlp = None

# Paths
MODEL_PATH = "models/water_model.h5"
SCALER_PATH = "models/scaler.pkl"
DATA_PATH = "data/water_potability.csv"

# Safe ranges (WHO-ish / heuristic) for explanation
SAFE_RANGES = {
    "ph": (6.5, 8.5),
    "Hardness": (60, 180),          # mg/L - approximate comfort range
    "Solids": (0, 500),             # TDS mg/L
    "Chloramines": (0.2, 4.0),      # mg/L typical disinfectant residual
    "Sulfate": (0, 250),            # mg/L
    "Conductivity": (0, 800),       # µS/cm (site-dependent)
    "Organic_carbon": (0, 5),       # mg/L
    "Trihalomethanes": (0, 80),     # µg/L
    "Turbidity": (0, 5)             # NTU
}

FEATURE_ORDER = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate',
                 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']

os.makedirs("models", exist_ok=True)


####### Helper: Train model if missing #######
def train_and_save_model(csv_path=DATA_PATH, model_path=MODEL_PATH, scaler_path=SCALER_PATH):
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"Dataset not found at {csv_path}. Please download and save water_potability.csv at that path.")
    print("Loading dataset for training:", csv_path)
    df = pd.read_csv(csv_path)
    # Fill missing values with median
    df = df.fillna(df.median())

    X = df.drop('Potability', axis=1)
    y = df['Potability']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.25, random_state=42, stratify=y
    )

    # Build neural network
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train
    early = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early], verbose=0)

    # Evaluate
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Model trained. Test accuracy: {acc:.3f}")

    # Save
    model.save(model_path)
    joblib.dump(scaler, scaler_path)
    print(f"Saved model -> {model_path}")
    print(f"Saved scaler -> {scaler_path}")
    return model, scaler


####### Helper: Load model and scaler (train if missing) #######
def load_model_and_scaler():
    if os.path.exists(MODEL_PATH) and os.path.exists(SCALER_PATH):
        print("Loading existing model and scaler...")
        model = load_model(MODEL_PATH)
        scaler = joblib.load(SCALER_PATH)
        print("Loaded model and scaler.")
    else:
        print("Model or scaler missing — training a new model (this may take a few minutes)...")
        model, scaler = train_and_save_model()
    return model, scaler

model, scaler = load_model_and_scaler()


####### Helper: parse user text to extract parameters #######
def extract_parameters(text):
    """
    Extracts known water features from free text.
    Returns a dict with keys in FEATURE_ORDER (values as floats or None if missing)
    Approach:
     - Try spaCy numbers + units if spaCy installed
     - Otherwise rely on regex patterns for each feature name
     - Also accepts JSON-like input or comma-separated key:value pairs
    """
    text = str(text).strip()
    parsed = {k: None for k in FEATURE_ORDER}

    # Try JSON-like input e.g., {"ph":7.2,"Turbidity":3}
    try:
        # sanitize simple JSON-like input
        if (text.startswith("{") and text.endswith("}")) or (":" in text and "," in text and any(k in text for k in ['ph','Turbidity','Hardness'])):
            # try direct json parse
            try:
                j = json.loads(text)
            except Exception:
                # replace single quotes if used
                j = json.loads(text.replace("'", "\""))
            for k, v in j.items():
                key = k.strip()
                if key in parsed:
                    try:
                        parsed[key] = float(v)
                    except:
                        continue
                else:
                    # try lowercase match
                    kl = key.lower()
                    for feat in parsed:
                        if feat.lower() == kl:
                            parsed[feat] = float(v)
            return parsed
    except Exception:
        pass

    # If spaCy available, do a best-effort extraction using entities + nearby tokens
    if nlp is not None:
        doc = nlp(text)
        # simple pattern: look for numeric tokens and nearby feature words
        nums = [token for token in doc if token.like_num]
        for num in nums:
            # check window around num for feature name
            window = doc[max(0, num.i-3): num.i+4]
            window_text = " ".join([t.text.lower() for t in window])
            for feat in parsed:
                if feat.lower() in window_text:
                    try:
                        parsed[feat] = float(num.text)
                    except:
                        pass

    # Regex fallback: look for patterns like "ph 7.2" or "turbidity: 3"
    # Pattern for each feature name (case-insensitive)
    for feat in FEATURE_ORDER:
        # build alternates for common name forms
        name_variants = [feat, feat.lower(), feat.capitalize()]
        # special-case 'ph' and 'Trihalomethanes' common synonyms
        if feat.lower() == 'ph':
            variants = [r"ph", r"pH", r"p\.h\.", r"p h"]
        elif feat.lower() == 'trihalomethanes':
            variants = [r"trihalomethanes", r"thm", r"trihalo"]
        else:
            variants = [re.escape(feat)]
        pat = re.compile(r"(?:%s)[^\d\-]*?([-+]?\d*\.?\d+)" % "|".join(variants), re.IGNORECASE)
        m = pat.search(text)
        if m:
            try:
                parsed[feat] = float(m.group(1))
            except:
                parsed[feat] = None

    # Another pass: find "value after feature" with colon or equals
    for feat in FEATURE_ORDER:
        if parsed[feat] is not None:
            continue
        pat2 = re.compile(r"(?i)%s\s*[:=]\s*([-+]?\d*\.?\d+)" % re.escape(feat))
        m2 = pat2.search(text)
        if m2:
            try:
                parsed[feat] = float(m2.group(1))
            except:
                parsed[feat] = None

    # Last resort: find standalone numbers if user asks only numbers in order
    numbers = re.findall(r"([-+]?\d*\.?\d+)", text)
    if all(v is not None for v in parsed.values()) and len(numbers) >= len(FEATURE_ORDER):
        # If user provided a bunch of numbers in order, map them
        for i, feat in enumerate(FEATURE_ORDER):
            parsed[feat] = float(numbers[i])

    return parsed


####### Helper: format parsed values to NN input vector #######
def build_input_vector(parsed):
    """
    Takes parsed dict (may contain None) -> returns np.array shape (1,9)
    missing values replaced with column medians from dataset scaler (or 0)
    """
    # If scaler has feature means/medians stored? We saved scaler only.
    # We'll use the original dataset medians as fallback:
    if not os.path.exists(DATA_PATH):
        # fallback zeros
        arr = np.zeros(len(FEATURE_ORDER))
        return arr.reshape(1, -1)
    df = pd.read_csv(DATA_PATH)
    df = df.fillna(df.median())
    arr = []
    for feat in FEATURE_ORDER:
        val = parsed.get(feat, None)
        if val is None:
            # use median
            median_val = float(df[feat].median())
            arr.append(median_val)
        else:
            arr.append(float(val))
    return np.array(arr).reshape(1, -1)


####### Helper: produce explanation text (generative-style, rule-based + data-driven) #######
def generate_explanation(parsed, prob):
    """
    Use feature deviations and safe ranges to create human-readable explanation.
    """
    lines = []
    prob_pct = float(prob)*100
    cls = "Drinkable" if prob >= 0.5 else "Not Drinkable"
    header = f"Prediction: **{cls}** (probability {prob_pct:.1f}%)"
    lines.append(header)
    lines.append("")  # blank line

    # Check each feature vs safe ranges
    issues = []
    for feat in FEATURE_ORDER:
        val = parsed.get(feat, None)
        if val is None:
            continue
        if feat in SAFE_RANGES:
            low, high = SAFE_RANGES[feat]
            if val < low:
                issues.append((feat, val, f"low (expected >= {low})"))
            elif val > high:
                issues.append((feat, val, f"high (expected <= {high})"))

    # Compose reasoning
    if len(issues) == 0:
        lines.append("✅ No strong violation of heuristic safe ranges detected.")
        lines.append("Recommendation: Routine monitoring and maintain current treatment.")
    else:
        lines.append("⚠️ Issues detected:")
        for feat, val, reason in issues:
            lines.append(f"- **{feat}** = {val} — {reason}.")
        # Provide recommended actions based on issues
        recs = set()
        feat_names = [f for f,_,_ in issues]
        if "Turbidity" in feat_names:
            recs.add("apply filtration / sedimentation and consider disinfection")
        if "ph" in feat_names:
            recs.add("adjust pH using lime (for low pH) or acid neutralization (for high pH)")
        if "Organic_carbon" in feat_names or "Trihalomethanes" in feat_names:
            recs.add("investigate organic pollution sources and optimize disinfection to reduce by-products")
        if "Conductivity" in feat_names or "Solids" in feat_names:
            recs.add("test for industrial/chemical contaminants and consider advanced treatment")
        lines.append("")
        lines.append("Suggested actions:")
        for r in recs:
            lines.append(f"- {r}.")

    # Confidence nuance
    if prob >= 0.9:
        lines.append("\n⚠️ High confidence in this prediction.")
    elif prob >= 0.75:
        lines.append("\nℹ️ Moderate-high confidence.")
    elif prob >= 0.5:
        lines.append("\nℹ️ Moderate confidence.")
    else:
        lines.append("\n⚠️ Lower probability of potability — treat as potentially unsafe until lab tests confirm.")

    # Return joined explanation
    return "\n".join(lines)


####### Core function: handle a user query #######
def handle_query(user_text):
    """
    1) Extract parameters from user_text
    2) Build input vector and scale
    3) Use neural net to predict probability
    4) Generate explanation and return structured output
    """
    parsed = extract_parameters(user_text)
    input_vec = build_input_vector(parsed)
    input_scaled = scaler.transform(input_vec)
    prob = float(model.predict(input_scaled)[0][0])
    classification = "Drinkable" if prob >= 0.5 else "Not Drinkable"
    explanation = generate_explanation(parsed, prob)

    # Build parsed-table for display
    parsed_display = {k: (v if v is not None else "N/A") for k, v in parsed.items()}
    parsed_df = pd.DataFrame.from_dict(parsed_display, orient='index', columns=['Value'])
    parsed_df.index.name = "Parameter"
    parsed_df = parsed_df.reset_index()

    return classification, prob, explanation, parsed_df


####### Gradio UI #######
def launch_gradio():
    examples = [
        "Is water with pH 7.2, Turbidity 3 and Conductivity 350 safe?",
        "ph:5.9, Hardness: 200, Turbidity: 8, Organic_carbon: 12",
        '{"ph":7.0, "Hardness":150, "Solids":300, "Turbidity":2}'
    ]

    with gr.Blocks() as demo:
        gr.Markdown("# 💧 Water Quality Chatbot — Generative AI Assistant")
        gr.Markdown(
            "Ask about a water sample in natural language (e.g., 'Is water with pH 6.2 and turbidity 8 safe?'). "
            "The assistant will parse values, predict potability, and generate a human-readable explanation."
        )
        textbox = gr.Textbox(lines=3, placeholder="Type your question here...", label="User Query")
        btn = gr.Button("Analyze")
        output_status = gr.Textbox(label="Classification (Drinkable / Not Drinkable)", interactive=False)
        output_prob = gr.Textbox(label="Predicted probability (0-1)", interactive=False)
        output_expl = gr.Textbox(label="Explanation and Recommendations", interactive=False)
        parsed_table = gr.Dataframe(headers=["Parameter", "Value"], label="Parsed Parameters", interactive=False)

        def on_click(user_text):
            cls, prob, expl, dfp = handle_query(user_text)
            return cls, f"{prob:.3f}", expl, dfp

        btn.click(on_click, inputs=[textbox], outputs=[output_status, output_prob, output_expl, parsed_table])
        gr.Examples(examples=examples, inputs=textbox)
        gr.Markdown("**Notes:** The model was trained on the Kaggle Water Potability dataset (global). For local policy use in Lesotho, validate with local lab data.")
    demo.launch()

if __name__ == "__main__":
    launch_gradio()


2025-10-30 22:37:57.395520: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-30 22:37:58.153293: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-30 22:38:00.272874: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Model or scaler missing — training a new model (this may take a few minutes)...


FileNotFoundError: Dataset not found at data/water_potability.csv. Please download and save water_potability.csv at that path.